In [ ]:
from google.colab import drive
import os
drive.mount('/content/drive/')

In [ ]:
%cd drive/MyDrive/YOLO_Custom

In [ ]:
# !git clone https://github.com/ultralytics/yolov5.git

In [ ]:
%cd yolov5

In [ ]:
!pip install -r requirements.txt

In [ ]:
!pip install albumentations

In [ ]:
! pip install --upgrade albumentations

In [ ]:
YOLO_PATH = '/content/drive/MyDrive/YOLO_Custom'
SAVE_PATH = '/content/drive/MyDrive/YOLO_Custom/Thermal_images/aug'

In [ ]:
from glob import glob

In [ ]:
img_path_list = glob(f'{YOLO_PATH}/Thermal_images/images/*.jpg')
print(len(img_path_list))

In [ ]:
import cv2
img_list = []
for img_path in img_path_list:
    image = cv2.imread(img_path)
    image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
    img_list.append(image)
img_list

In [ ]:
bbox_path_list = glob(f'{YOLO_PATH}/Thermal_images/labels/*.txt')
len(bbox_path_list)

In [ ]:
import numpy as np
bbox_list = []
for bbox_path in bbox_path_list:
    with open(bbox_path, 'r') as f:
        bbox = f.read()
        bbox = bbox.split('\n')
        bboxes = [x.split() for x in bbox]
        real_box = []
        for b in bboxes:
            try:
                temp = b[0]
                temp_boxes =  b[1:]
                temp_boxes.append(temp)
                real_box.append(temp_boxes)
            except:
                pass
        bbox_np = np.array(real_box).astype(float)
        bbox_list.append(bbox_np)
bbox_list

In [ ]:
import albumentations as A
import cv2
transform = A.Compose([
    A.Rotate(limit=20, p=0.2, border_mode=cv2.BORDER_CONSTANT),
    A.HorizontalFlip(p=0.8),
], bbox_params=A.BboxParams(format='yolo')
)

In [ ]:
aug_image_list = img_list.copy()
aug_bbox_list = bbox_list.copy()
idx = 0
for image, bboxes in zip(img_list, bbox_list):
    for i in range(3):
        try:
            augmentations = transform(image=image, bboxes=bboxes)
            augmented_img = augmentations['image']
            augmented_bboxes = augmentations['bboxes']
            aug_image_list.append(augmented_img)
            aug_bbox_list.append(augmented_bboxes)
            idx+=1
            print('success', idx)
        except:
            pass

In [ ]:
aug_bbox_list[0]

In [ ]:
aug_bbox_list[300], aug_image_list[300]

In [ ]:
from sklearn.model_selection import train_test_split

train_img_list, val_img_list = train_test_split(img_path_list, test_size=.15, random_state=8)

In [ ]:
with open(f'{YOLO_PATH}/Thermal_images/train.txt', 'w') as f:
    f.write('\n'.join(train_img_list) + '\n')
with open(f'{YOLO_PATH}/Thermal_images/val.txt', 'w') as f:
    f.write('\n'.join(val_img_list) + '\n')

In [ ]:
import yaml

In [ ]:
with open(f'{YOLO_PATH}/Thermal_images/data.yaml', 'r') as f:
    data = yaml.load(f)
print(data)

In [ ]:
%cd

In [ ]:
data['train'] = '../Thermal_images/train.txt'
data['val'] = '../Thermal_images/val.txt'

with open(f'{YOLO_PATH}/data.yaml', 'w') as f:
    yaml.dump(data, f)

In [ ]:
import os
os.chdir(YOLO_PATH + '/yolov5')
os.getcwd()

In [ ]:
!python train.py --img 416 --batch 8 --epochs 100 --data /content/drive/MyDrive/YOLO_Custom/Thermal_images/data.yaml --cfg ./models/yolov5s.yaml --weights yolov5s.pt --name Thermal_yolov5s_results

In [ ]:
%load_ext tensorboard
%tensorboard --logdir /content/drive/MyDrive/YOLO_Custom/yolov5/runs/

In [ ]:
from IPython.display import Image
import os

val_img_path = train_img_list[117]

!python detect.py --weights /content/drive/MyDrive/YOLO_Custom/yolov5/runs/train/Thermal_yolov5s_results/weights/best.pt --img 416 --conf 0.3 --source {val_img_path}
Image(os.path.join('/content/drive/MyDrive/YOLO_Custom/yolov5/runs/detect/exp', os.path.basename(val_img_path)))

In [ ]:
!pip install r requirements.txt